# Reading data from Excel

Let's get some data. Download [Sample Superstore Sales .xls file](https://community.tableau.com/docs/DOC-1236) or [my local copy](https://github.com/parrt/msan692/raw/master/data/SampleSuperstoreSales.xls) and open it in Excel to see what it looks like.

Data of interest that we want to process in Python often comes in the form of an Excel spreadsheet, but the data is in a special non-text-based (binary) format that we can't read directly:

In [1]:
with open('data/SampleSuperstoreSales.xls', "rb") as f:
    txt = f.read()
    print(txt[0:100])

b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00/\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x13\x17\x00\x00\x01\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00b\x00\x00\x00\xe3\x00\x00\x00d\x01\x00\x00\xe5\x01\x00\x00f\x02\x00\x00'


## Converting Excel files with csvkit

There's a really useful tool kit called `csvkit`, which you can install with:

In [2]:
! pip install -q -U csvkit

Then, the following command works without having to run or even own Excel on your laptop (but you might get lots of warnings):

In [6]:
! in2csv data/SampleSuperstoreSales.xls > /tmp/t.csv

In [7]:
! head -3 /tmp/t.csv  # show 3 lines of t.csv

Row ID,Order ID,Order Date,Order Priority,Order Quantity,Sales,Discount,Ship Mode,Profit,Unit Price,Shipping Cost,Customer Name,Province,Region,Customer Segment,Product Category,Product Sub-Category,Product Name,Product Container,Product Base Margin,Ship Date
1.0,3.0,2010-10-13,Low,6.0,261.54,0.04,Regular Air,-213.25,38.94,35.0,Muhammed MacIntyre,Nunavut,Nunavut,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,0.8,2010-10-20
49.0,293.0,2012-10-01,High,49.0,10123.02,0.07,Delivery Truck,457.81,208.16,68.02,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Appliances,"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators",Jumbo Drum,0.58,2012-10-02


You might be wondering why we would need a commandline version that converts Excel files to CSV.  Certainly, as we see in the next section, we can use pandas.  The difference is that using the command line means we don't have to write software, which is much more involved than simply executing a few lines on the commandline. For example, we could easily use a shell `for` loop to convert multiple Excel files to csv without starting a Python development environment.

## Reading Excel files with Pandas

The easiest way to read Excel files with Python is to use Pandas:

In [12]:
import pandas as pd
table = pd.read_excel("data/SampleSuperstoreSales.xls")
table.head(3)

,Row ID,Order ID,Order Date,Order Priority,Order Quantity,Sales,Discount,Ship Mode,Profit,Unit Price,...,Customer Name,Province,Region,Customer Segment,Product Category,Product Sub-Category,Product Name,Product Container,Product Base Margin,Ship Date
0,1,3,2010-10-13,Low,6,261.54,0.04,Regular Air,-213.2500,38.94,...,Muhammed MacIntyre,Nunavut,Nunavut,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,0.80,2010-10-20
1,49,293,2012-10-01,High,49,10123.02,0.07,Delivery Truck,457.8100,208.16,...,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Appliances,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Jumbo Drum,0.58,2012-10-02
2,50,293,2012-10-01,High,27,244.57,0.01,Regular Air,46.7075,8.69,...,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Binders and Binder Accessories,"Cardinal Slant-D® Ring Binder, Heavy Gauge Vinyl",Small Box,0.39,2012-10-03


# CSV data

Grab the CSV version of the Excel file [SampleSuperstoreSales.csv](https://github.com/parrt/msan692/raw/master/data/SampleSuperstoreSales.csv) we've been playing with. You already know how to load CSV with pandas, but sometimes it's reasonable to process the CSV at a lower level, without getting a big data frame back. For example, we might want to simply process the CSV file line by line if it is truly huge.

In your project, you dealt with very simple CSV, but it can get much more complicated so let's take a quick look how to use the `csv` package. CSV files can get complicated when we need to embed a comma. One such case from the above file shows how fields with commas get quoted:

```
"Eldon Base for stackable storage shelf, platinum"
```

What happens when we want to encode a quote? Well, somehow people decided that `""` double quotes was the answer (not!) and we get fields encoded like this:

```
"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators"
```

The good news is that Python's `csv` package knows how to read Excel-generated files that use such encoding. Here's a sample script that reads such a file into a list of lists:

In [3]:
import sys
import csv

table_file = "data/SampleSuperstoreSales.csv"
with open(table_file, "r") as csvfile:
    f = csv.reader(csvfile, dialect='excel')
    data = []
    for row in f:
        data.append(row)

print(data[:6])

[['Row ID', 'Order ID', 'Order Date', 'Order Priority', 'Order Quantity', 'Sales', 'Discount', 'Ship Mode', 'Profit', 'Unit Price', 'Shipping Cost', 'Customer Name', 'Province', 'Region', 'Customer Segment', 'Product Category', 'Product Sub-Category', 'Product Name', 'Product Container', 'Product Base Margin', 'Ship Date'], ['1', '3', '10/13/10', 'Low', '6', '261.54', '0.04', 'Regular Air', '-213.25', '38.94', '35', 'Muhammed MacIntyre', 'Nunavut', 'Nunavut', 'Small Business', 'Office Supplies', 'Storage & Organization', 'Eldon Base for stackable storage shelf, platinum', 'Large Box', '0.8', '10/20/10'], ['49', '293', '10/1/12', 'High', '49', '10123.02', '0.07', 'Delivery Truck', '457.81', '208.16', '68.02', 'Barry French', 'Nunavut', 'Nunavut', 'Consumer', 'Office Supplies', 'Appliances', '1.7 Cubic Foot Compact "Cube" Office Refrigerators', 'Jumbo Drum', '0.58', '10/2/12'], ['50', '293', '10/1/12', 'High', '27', '244.57', '0.01', 'Regular Air', '46.71', '8.69', '2.99', 'Barry French'

Or add to a numpy `array`:

In [4]:
import numpy as np
np.array(data)

array([['Row ID', 'Order ID', 'Order Date', ..., 'Product Container',
        'Product Base Margin', 'Ship Date'],
       ['1', '3', '10/13/10', ..., 'Large Box', '0.8', '10/20/10'],
       ['49', '293', '10/1/12', ..., 'Jumbo Drum', '0.58', '10/2/12'],
       ...,
       ['7906', '56550', '4/8/11', ..., 'Small Pack', '0.41', '4/10/11'],
       ['7907', '56550', '4/8/11', ..., 'Small Box', '0.56', '4/9/11'],
       ['7914', '56581', '2/8/09', ..., 'Medium Box', '0.65', '2/11/09']],
      dtype='<U98')

## Reading CSV into Pandas Data frames

(Those who have taken exploratory data analysis, will already know this part.)

In the end, the easiest way to deal with loading CSV files is probably with [Pandas](http://pandas.pydata.org/), even if you need the data in a different data structure. For example, to load our sales CSV, we don't even have to manually open and close a file:

In [10]:
import pandas # revisiting EDA pandas lectures...
df = pandas.read_csv("data/SampleSuperstoreSales.csv")
df.head(3)

,Row ID,Order ID,Order Date,Order Priority,Order Quantity,Sales,Discount,Ship Mode,Profit,Unit Price,...,Customer Name,Province,Region,Customer Segment,Product Category,Product Sub-Category,Product Name,Product Container,Product Base Margin,Ship Date
0,1,3,10/13/10,Low,6,261.54,0.04,Regular Air,-213.25,38.94,...,Muhammed MacIntyre,Nunavut,Nunavut,Small Business,Office Supplies,Storage & Organization,"Eldon Base for stackable storage shelf, platinum",Large Box,0.80,10/20/10
1,49,293,10/1/12,High,49,10123.02,0.07,Delivery Truck,457.81,208.16,...,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Appliances,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Jumbo Drum,0.58,10/2/12
2,50,293,10/1/12,High,27,244.57,0.01,Regular Air,46.71,8.69,...,Barry French,Nunavut,Nunavut,Consumer,Office Supplies,Binders and Binder Accessories,"Cardinal Slant-D® Ring Binder, Heavy Gauge Vinyl",Small Box,0.39,10/3/12


Pandas hides all of the details. I also find that pulling out columns is nice with pandas. Here's how to print the customer name column:

In [11]:
df['Customer Name'].head()

0    Muhammed MacIntyre
1          Barry French
2          Barry French
3         Clay Rozendal
4        Carlos Soltero
Name: Customer Name, dtype: object

In [7]:
df.Profit.head()

0    -213.25
1     457.81
2      46.71
3    1198.97
4      30.94
Name: Profit, dtype: float64

### Exercise

From the sales CSV file, use pandas to read in the data and multiply the `Order Quantity` and `Unit Price` columns to get a new column.

In [8]:
df = pandas.read_csv("data/SampleSuperstoreSales.csv")
(df['Order Quantity']*df['Unit Price']).head()

0      233.64
1    10199.84
2      234.63
3     5879.70
4      413.82
dtype: float64